# 3.2B Construcción y transformación de datos
Pandalytics - Equipo 1
* **A00832444** | Andrea Garza  
* **A01197991** | Hiram Maximiliano Muñoz Ramírez  
* **A00517124** | Erick Orlando Hernández Vallejo  
* **A01197655** | Raúl Isaí Murillo Alemán   
* **A01235692** | David Gerardo Martíne Hidrogo 

In [1]:
import pandas as pd
import numpy as np

paint_df = pd.read_feather('data/analisis_consumo_pintura.feather')
paint_square_meters_df = pd.read_feather('data/pintura_metros_cuadrados_reales.feather')
paint_performance_df = pd.read_feather('data/rendimiento_pintura.feather')
production_df = pd.read_feather('data/production.feather')

coating_df = pd.read_feather('data/pinturas_revestidos_jul20_ago23.feather')
defects_df=pd.read_feather('data/defects.feather')

## 1 _Pipeline_ de datos
Se definió una _pipeline_ de procesamiento de datos en un _script_ de Python independiente, con el cuál se tiene un proceso replicable para llegar a una base de datos limpia. Esta _pipeline_ consta de 6 diferentes pasos:

- Importación de datos
- Concatenación de resúmenes de producción
- Separación de analisis de líneas de producción y sus defectos
- Selección y renombramiento de columnas
- Eliminación de columnas adicionales
- Conversión a formato _Feather_.

A continuación se muestra un resumen de todo el proceso de la _pipeline_ de datos:

### 1.1 Importación de datos
La importación de datos consiste en importar todos los datos relevantes de los archivos de excel proveídos por Ternium. Hay un total de 12 archivos diferentes.Cada uno de estos archivos contiene varias _worksheets_ dentro. Para la mayoría, solamente necesitamos importar una de estas. Solamente el archivo de Análisis de consumo de pintura requirió importar mas _worksheets_:

- Pinturas y revestidos
    - Pintado 1 UNI Agosto
- Análisis de consumo de pintura
    - Base de datos por pintura
    - Metros
    - Rendimientos INDU
- 10 archivos de resúmenes de producción
    - Resúmen producción
 
### 1.2 Concatenación de resúmenes de producción
Para unificar toda la información de resúmenes de producción, concatenamos todos estas tablas, para así acabar con una base de datos más cohesiva y extensiva con la cual trabajar. Se ignoro el índice original de las 10 tablas, ya que realmente no tenían ningún significado.

### 1.3 Separación de analisis de líneas de producción y sus defectos
Los resúmenes de producción contienen un grupo de columnas que se repiten 5 veces. Estás columnas representan hasta 5 posibles defectos en cada línea de producción. Por lo tanto, estas columnas representan una relación uno a muchos entre una línea de producción y sus posibles defectos. Se definió una función que separa estos grupos de 5 columnas hacia una nueva tabla de defectos, con cada defecto recibiendo una referencia hacia su respectiva línea de producción. Finalmente, se eliminaron las columnas de defectos de la tabla original, ya que ahora se pueden referenciar por medio de una operación de merge con la nueva tabla de defectos.

### 1.4 Selección y renombramiento de columnas
Se seleccionaron columnas relevantes de cada base de datos para dejar en la base de datos. Estas columnas fueron renombradas para mejor documentar su propósito. A continuación se tiene una lista de las columnas renombradas para cada base de datos, así como las razones por las cuales las seleccionamos.

#### 1.4.1 Pinturas y revestidos
- **'Denominación objeto' por 'production_line':** Posible identificador para cada línea de producción
- **'Material' por 'paint_id'**: Pintura usada, posible identificador para pinturas
- **'Texto breve de material' por 'paint_name'**: Nombre de la pintura usada, posible identificador para pinturas
- **'Valor var.' por 'monetary_value_usd'**: Valor monetario en dólares, útil para calcular costos
- **'Ctd.total reg.' por 'total_liters_used'**: Litros usados, dato clave
- **'Planta' por 'production_plant'**: Identificación de planta, para filtrar datos
- **'Proveedor' por 'supplier'**: Proveedor, para análisis de datos agrupados
- **'Registrado' por 'date'**: Fecha de los datos, dato clave
- **'Hora' por 'hour'**: Hora de los datos, dato clave
- **'Precio' por 'price_per_liter'**: Precio de cada litro, útil para calcular costos

#### 1.4.2 Análisis de consumo de pintura
- **'Linea' por 'production_line'**: Línea de producción
- **'Mes' por 'month'**: Mes del consumo de pintura, dato clave
- **'Mes num' por 'month_number'**: Número del mes
- **'Pintura' por 'paint'**: Pintura, posible identificador para pinturas
- **'Real' por 'real_consumption'**: Consumo real, dato clave
- **'Teo' por 'theoretical_consumption'**: Consumo tórico, dato clave
- **'Dif' por 'consumption_difference'**: Diferencia de consumos, dato clave
- **'Rendimeinto Std' por 'average_yield'**: Rendimiento promedio, dato clave
- **'Rendimeinto Real' por 'real_yield'**: Rendimiento real, dato clave
- **'Diferencia de Rendimiento' por 'yield_difference'**: Diferencia de rendimientos, dato clave
- **'Metros cuadrados reales' por 'real_produced_square_meters'**: Metros cuadrados reales producidos, dato clave

#### 1.4.3 Rendimientos de pinturas por metro cuadrado
- **'Linea' por 'production_line'**: Identificador para cada línea de producción
- **'Mes' por 'month'**: Mes del dato
- **'Num mes' por 'month_number'**: Numero del mes
- **'Pintura' por 'paint_name'**: Nombre de la pintura
- **'Metros cuadrados reales (m2)' por 'real_square_meters'**: Metros cuadrados de rendimiento de la pintura

#### 1.4.4 Rendimientos de pinturas metros cuadrados por litro
- **'Pintura' por 'paint_name'**: Nombre de la pintura
- **'Clave' por 'paint_code'**: Identificador de la pintura
- **'Rendimiento Canning [m2/L]' por  'paint_performance_m2/l'**: Rendimiento de metros cuadrados por litro

#### 1.4.5 Resumen de producción
- **'Linea' por 'production_line'**: Posible identificador para cada línea de producción
- **'Material Entrada' por 'input_material_code'**: Id del material de entrada, dato clave
- **'Material Salida' por 'output_material_code'**: Id del material de salida, dato clave
- **'Fecha Inicio' por 'start_date'**: Fecha de inicio de la producción, dato clave
- **'Fecha Fin' por 'end_date'**: Fecha de fin de la producción, dato clave
- **'Cliente' por 'client_name'**: Cliente, posible agrupación
- **'Código Clear Inf' por 'inferior_clear_code'**: Código del _clear_ inferior
- **'Código Clear Sup' por 'superior_clear_code'**: Código del _clear_ superior
- **'Ancho 1' por 'width1_mm'**: Ancho 1, dato clave
- **'Ancho 2' por 'width2_mm'**: Ancho 2, dato clave
- **'Ancho 3' por 'width3_8mm'**: Ancho 3, dato clave
- **'Ancho' por 'width_mm'**: Ancho promedio, dato clave
- **'Espesor 1' por 'thickness1_mm'**: Espesor 1, dato clave
- **'Espesor 2' por 'thickness2_mm'**: Espesor 2, dato clave
- **'Espesor 3' por 'thickness3_mm'**: Espesor 3, dato clave
- **'Espesor' por 'thickness_mm'**: Espesor promedio, dato clave
- **'Peso Entrada' por 'input_weight_kg'**: Peso de entrada, dato clave
- **'Peso' por 'weight_kg'**: Peso de salida, dato clave
- **'Largo' por 'length_m'**: largo total, dato clave
- **'Color Inferior' por 'inferior_color_code'**: Código del color inferior
- **'Color Superior' por 'superior_color_code'**: Código del color superior
- **'Primer Superior' por 'superior_primer_code'**: Código del _primer_ superior
- **'Primer Inferior' por 'inferior_primer_code'**: Código del _primer_ inferior
- **'Ruta Teórica' por 'route'**: Ruta teórica de la producción, nos sirve para filtrar las líneas de producción de UNI

#### 1.4.6 Defectos de producción
- **'Codigo defecto' por 'defect_code'**: Posible identificador para tipo de defecto
- **'Defecto' por 'defect_name'**: Nombre de defecto
- **'Ubicacion' por 'location'**: Ubicación en la línea donde sucedió el defecto
- **'Es Contencion' por 'is_containment'**: Si el defecto fue contención de un suceso
- **'Es Prevencion' por 'is_preventive'**: Si el defecto fue prevención de un riesgo
- **'Intensidad' por 'intensity'**: Intensidad del suceso
- **'Cara' por 'face'**: Cara donde sucedió el defecto
- **'Lado' por 'side'**: Lado donde sucedió el defecto
- **'Frecuencia' por 'frequency'**: Frecuencia del defecto
- **'Fecha Registro' por 'register_date'**: Fecha del suceso

### 1.5 Eliminación de columnas adicionales
Una vez seleccionadas y renombradas las columnas relevantes, se excluyeron de las tablas todas las columnas adicionales.

### 1.6 Conversión a formato _feather_
_Feather_ es un format de archivos que permite almacenar tablas o _dataframes_ de una manera eficiente y agnostica al lenguaje. Para el reto, es mucho mas eficiente trabajar con los datos en formato _Feather_ que directamente con los archivos de Excel, por lo cual se decidió utilizar este formato para almacenar los artefactos generados por la _pipeline_ de datos.

El proceso de escribir una tabla a formato _Feather_ automáticamente infiere los tipos de todas las columnas, convirtiendo las columnas númericas a sus respectivos tipos, p. ej. '1.02' a tipo _float_ y '232' a tipo _int_. Cualquier otra columna será convertida a tipo _object_, con las cuales se debe llevar a cabo una conversión manual. Esta conversión es realizada en secciones posteriores de este documento.

## 2 Filtrado de datos
Para cuestiones del reto, solamente es de interés trabajar con datos que sean de la planta Ternium Universidad. Por lo tanto, se necesita filtrar las tablas a que únicamente incluyan valores provenientes de ahi.

### 2.1 Pinturas y revestidos
Para la tabla de pinturas y revestidos, la columna `production_plant` indica a cual planta pertenece cada línea de producción. Por lo tanto, se filtraron valores diferentes a 'Uni'.

In [2]:
coating_df = coating_df[coating_df['production_plant'] == 'Uni']
coating_df                                                                                            

,production_line,paint_id,paint_name,monetary_value_usd,total_liters_used,production_plant,supplier,date,hour,price_per_liter
2,Pintado 1 Prod. Univ,U0222_AKZO,0222-AZUL RAL-5003 FC,5976.52,189.250,Uni,AKZO,24.07.2020,00:12:52,31.580026
35,Pintado 2 Prod. Univ,I3329_AKZO,0318-TOPAZ METALLIC 14H2593,3314.07,189.267,Uni,AKZO,24.07.2020,22:28:26,17.510026
36,Pintado 2 Prod. Univ,I3329_AKZO,0318-TOPAZ METALLIC 14H2593,3314.07,189.267,Uni,AKZO,24.07.2020,21:28:35,17.510026
44,Pintado 1 Prod. Univ,I3052_VALS,0121-BLANCO STD KYNAR,3126.00,200.000,Uni,VALS,25.07.2020,14:00:07,15.630000
176,Pintado 1 Prod. Univ,I3052_VALS,0121-BLANCO STD KYNAR,2625.84,168.000,Uni,VALS,25.07.2020,14:32:47,15.630000
...,...,...,...,...,...,...,...,...,...,...
211698,Pintado 1 UNI,I0402_VALS,0400-GRIS FONDO,-423.00,-75.000,Uni,VALS,18.08.2023,00:00:00,5.640000
211699,Pintado 1 UNI,I4078_VALS,WHITE KRYSTAL KOTE,-896.00,-70.000,Uni,VALS,18.08.2023,00:00:00,12.800000
211701,Pintado 1 UNI,I0100_VALS,0102-BLANCO STD,-1170.00,-200.000,Uni,VALS,18.08.2023,00:00:00,5.850000
211704,Pintado 1 UNI,I1270_BECK,0079-BECKRYPRIM 246,-1470.00,-200.000,Uni,BECK,18.08.2023,00:00:00,7.350000


### 2.2 Resumen de producción
Para la tabla de resumen de producción, la columna `route` contiene la ruta teoríca que cada pintura sigue. Para verificar si estos son datos relevantes al caso, fue decidido que se filtraria esta columna con base en las partes de cada ruta. Cada ruta esta estructurada de manera que dice 'planta-planta-planta-planta', separados por guiones. Por lo tanto, se separaron todos los valores y se busco que tuvieran el valor 'UNI' dentro.

In [3]:
routes = production_df['route'].apply(lambda route: all(map(lambda value: 'UNI' in value, route.split('-'))))
production_df = production_df[routes]
production_df

,input_material_code,output_material_code,start_date,end_date,client_name,inferior_clear_code,superior_clear_code,width1_mm,width2_mm,width3_mm,...,thickness3_mm,thickness_mm,input_weight_kg,weight_kg,length_m,inferior_color_code,superior_color_code,superior_primer_code,inferior_primer_code,route
0,4A058886UP100,4A058886UP101,2023-03-31 22:57:32,2023-03-31 23:42:50,CONSTRUCCIONES REFRIGERADAS SA,None,None,1223.0,1223.0,1223.0,...,0.464,0.4640,5290,5205,1175,None,None,None,None,G4_UNI-P1_UNI-LAB_UNI-EMBP1_UNI
1,2B126983UG400,2B126983UP100,2023-03-31 23:42:50,2023-04-01 00:03:27,CONSORCIO MINERO BENITO JUAREZ PEÑA COLORADA S...,None,None,1221.0,1221.0,1221.0,...,0.739,0.7390,20660,6860,954,0121-BLANCO STD KYNAR,0226-BREATHTAKING BLUE FC,0038-PRIMER 917,0038-PRIMER 917,G4_UNI-P1_UNI-LAB_UNI-EMBP1_UNI
2,2B126983UG400,2B126983UP101,2023-04-01 00:03:27,2023-04-01 00:23:34,CONSORCIO MINERO BENITO JUAREZ PEÑA COLORADA S...,None,None,1221.0,1221.0,1221.0,...,0.739,0.7390,20660,6845,952,0121-BLANCO STD KYNAR,0226-BREATHTAKING BLUE FC,0038-PRIMER 917,0038-PRIMER 917,G4_UNI-P1_UNI-LAB_UNI-EMBP1_UNI
3,2B126983UG400,2B126983UP102,2023-04-01 00:23:34,2023-04-01 00:43:19,CONSORCIO MINERO BENITO JUAREZ PEÑA COLORADA S...,None,None,1221.0,1221.0,1221.0,...,0.739,0.7390,20660,6500,904,0121-BLANCO STD KYNAR,0226-BREATHTAKING BLUE FC,0038-PRIMER 917,0038-PRIMER 917,G4_UNI-P1_UNI-LAB_UNI-EMBP1_UNI
4,2B126984UG400,2B126984UP100,2023-04-01 00:43:19,2023-04-01 00:53:22,CONSORCIO MINERO BENITO JUAREZ PEÑA COLORADA S...,None,None,1222.0,1222.0,1222.0,...,0.759,0.7590,11525,3165,427,0121-BLANCO STD KYNAR,0226-BREATHTAKING BLUE FC,None,None,G4_UNI-P1_UNI-LAB_UNI-EMBP1_UNI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17644,3B530354UG400,3B530354UP201,2023-08-31 21:19:28,2023-08-31 21:35:48,INDUSTRIAS ACROS WHIRLPOOL,0028-CLEAR P/ESPUMA,None,919.0,919.0,919.0,...,0.383,0.3833,15753,4183,1494,None,0104-BLANCO SUPERMATIC,0021-UNIVERSAL PRIMER,None,G4_UNI-P2_UNI-LAB_UNI-EMBP2_UNI
17645,3B530354UG400,3B530354UP202,2023-08-31 21:35:48,2023-08-31 21:51:53,INDUSTRIAS ACROS WHIRLPOOL,0028-CLEAR P/ESPUMA,None,919.0,919.0,919.0,...,0.383,0.3833,15753,4113,1468,None,0104-BLANCO SUPERMATIC,0021-UNIVERSAL PRIMER,None,G4_UNI-P2_UNI-LAB_UNI-EMBP2_UNI
17646,3B530354UG400,3B530354UP203,2023-08-31 21:51:53,2023-08-31 22:07:26,INDUSTRIAS ACROS WHIRLPOOL,0028-CLEAR P/ESPUMA,None,919.0,919.0,919.0,...,0.383,0.3833,15753,3963,1415,None,0104-BLANCO SUPERMATIC,0021-UNIVERSAL PRIMER,None,G4_UNI-P2_UNI-LAB_UNI-EMBP2_UNI
17647,4A250820UN100,4A250820UP204,2023-08-31 22:07:26,2023-08-31 22:20:40,INDUSTRIAS ACROS WHIRLPOOL,0028-CLEAR P/ESPUMA,None,918.0,918.0,918.0,...,0.372,0.3723,6750,3343,1248,None,0104-BLANCO SUPERMATIC,0021-UNIVERSAL PRIMER,None,G4_UNI-P2_UNI-LAB_UNI-EMBP2_UNI


## 3 Verificación de inconsistencias en los datos

### 3.1 Análisis de consumo de pintura
Primero, para todas las columnas de tipo objeto de la base de datos de análisis de consumo de pintura se convirtieron a sus respectivos tipos.
En la tabla principal, hay dos:
- production_line: solo tiene dos valores, por lo que es **categoríca**
- month: es el mes textual, por lo que es **categoríca**
- paint: es el identificador de la pintura, por lo que es un **string**

En la tabla de rendimiento por metro cuadrado, hay 3:
- production_line: solo tiene dos valores, por lo que es **categoríca**
- month: es el mes textual, por lo que es **categoríca**
- paint_name: es el identificador de la pintura, por lo que es un **string**

En la tabla de rendimiento de metro cuadrado por litro, hay 2:
- paint_name: el nombre de la pintura, por lo que es un **string**
- paint_code: el identificador de la pintura, por lo que es un **string**

In [4]:
paint_df['production_line'] = paint_df['production_line'].astype('category')
paint_df['month'] = paint_df['month'].astype('category')
paint_df['paint'] = paint_df['paint'].astype('string')
paint_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   production_line              566 non-null    category
 1   month                        566 non-null    category
 2   month_number                 566 non-null    int64   
 3   paint                        566 non-null    string  
 4   real_consumption             566 non-null    float64 
 5   theoretical_consumption      566 non-null    float64 
 6   consumption_difference       566 non-null    float64 
 7   average_yield                566 non-null    float64 
 8   real_yield                   566 non-null    float64 
 9   yield_difference             566 non-null    float64 
 10  real_produced_square_meters  566 non-null    float64 
dtypes: category(2), float64(7), int64(1), string(1)
memory usage: 41.4 KB


In [5]:
paint_square_meters_df['production_line'] = paint_square_meters_df['production_line'].astype('category')
paint_square_meters_df['month'] = paint_square_meters_df['month'].astype('category')
paint_square_meters_df['paint_name'] = paint_square_meters_df['paint_name'].astype('string')
paint_square_meters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   production_line     620 non-null    category
 1   month               620 non-null    category
 2   month_number        620 non-null    int64   
 3   paint_name          620 non-null    string  
 4   real_square_meters  620 non-null    float64 
dtypes: category(2), float64(1), int64(1), string(1)
memory usage: 16.2 KB


In [6]:
paint_performance_df['paint_name'] = paint_performance_df['paint_name'].astype('string')
paint_performance_df['paint_code'] = paint_performance_df['paint_code'].astype('string')
paint_performance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1997 entries, 0 to 1996
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   paint_name              1997 non-null   string 
 1   paint_code              1997 non-null   string 
 2   paint_performance_m2/l  1958 non-null   float64
dtypes: float64(1), string(2)
memory usage: 46.9 KB


#### 3.1.1 Diferencia entre consumo real y teórico

Verificamos que la columna de diferencia de consumo se calculara correctamente mediante una serie que contiene la resta del consumo real menos el consumo teórico y comparamos esta diferencia con la de la columna del archivo para verificar si todos los registro son correctos.

In [7]:
calculated_consumption_difference = paint_df['real_consumption'] - paint_df['theoretical_consumption']
if paint_df['consumption_difference'].equals(calculated_consumption_difference):
    print('La columna "consumption_difference" se calculó correctamente.')
else:
    print('La columna "consumption_difference" no se calculó correctamente.')

La columna "consumption_difference" se calculó correctamente.


#### 3.1.2 Metros cuadrados reales

Verificamos que los metros cuadrados reales se calcularan correctamente en el dataframe de análisis de consumo de pintura por medio de un dataframe generado por mmedio de un merge con el dataframe de los metros cuadrados de cada pintura para localizar aquellos puntos en los que tanto la línea de producción, el mes y el nombre de la pintura coincidan para comparar los metros cuadrados de cada dataframe

In [8]:
paint_merged_df = pd.merge(paint_df, paint_square_meters_df, how='left', left_on=['production_line', 'month', 'paint'], right_on=['production_line', 'month', 'paint_name'])
if paint_merged_df['real_produced_square_meters'].equals(paint_merged_df['real_square_meters']):
    print('Los metros cuadrados se calcularon correctamente.')
else:
    print('Los metros cuadrados no se calcularon correctamente.')

Los metros cuadrados se calcularon correctamente.


#### 3.1.3 Rendimiento Estandar VS Rendimiento Real

Se calculó el nuevo rendimiento std utilizando el promedio del rendimientos por pintura, en vez de utilizar el primer valor encontrado en la hoja de rendimientos. Después, se agrego una columna new_average_yield (nuevo rendimiento std) dentro pintura_df. 

In [9]:
new_performance = paint_performance_df[['paint_name','paint_performance_m2/l']].groupby('paint_name', as_index=False).mean()
paint_df = paint_df.merge(new_performance, left_on='paint', right_on='paint_name')
paint_df.rename(columns = { 'paint_performance_m2/l': 'new_average_yield'}, inplace=True)
paint_df.drop(['paint_name'], axis=1, inplace=True)
paint_df

,production_line,month,month_number,paint,real_consumption,theoretical_consumption,consumption_difference,average_yield,real_yield,yield_difference,real_produced_square_meters,new_average_yield
0,Pintado 1,Abril,4,0226-BREATHTAKING BLUE FC,203.25,185.69356,17.55644,24.146982,22.061200,-2.085781,4483.939000,24.146982
1,Pintado 1,Abril,4,0121-BLANCO STD KYNAR,1412.75,1690.59626,-277.84626,20.997375,27.639634,6.642259,39047.893000,22.047244
2,Pintado 1,Abril,4,0919-ROJO TERNIUM CR,487.36,472.13485,15.22515,22.572178,6.384172,-16.188007,3111.390000,22.572178
3,Pintado 1,Abril,4,1487-AMERICAN STERLING III,760.00,729.61405,30.38595,26.404494,24.706582,-1.697913,18777.002000,24.719101
4,Pintado 1,Abril,4,0435-APOLLO GRAY KRYSTAL KOTE,6906.00,7242.44193,-336.44193,26.666667,27.114563,0.447896,187253.169513,26.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
561,Pintado 2,Agosto,8,0099-CLEAR P/ESPUMA ROHS,5866.75,6436.95611,-570.20611,40.944882,50.031994,9.087112,293525.200300,39.370079
562,Pintado 2,Agosto,8,0075-CLEAR LAUNDRY BACKER ROHS,1444.00,995.55940,448.44060,77.165354,60.218956,-16.946398,86956.173000,77.165354
563,Pintado 2,Agosto,8,0023-CLEAR EPOXICO P/ESPUMA,1073.25,1184.15414,-110.90414,39.370079,52.568644,13.198565,56419.297265,44.291339
564,Pintado 2,Agosto,8,1028-CLEAR BLUE EPOXY,200.00,191.27243,8.72757,42.519685,46.948690,4.429005,9389.738000,40.944882


Se calculo el rendimiento real usando la fórmula que ternium nos proporcionó (metros cuadrados reales / rendimiento real), en donde se almacena en una variable. Después, generamos la diferencia entre lo que nosotros calculamos con los datos dentro de la base de datos. Finalmente, el código filtra los datos y verifica si hay diferencias mayores a 0 y de ser así las imprime, si no imprime que no hay.

In [10]:
#Rendimiento real
calculated_real_yield =  paint_df['real_produced_square_meters'] / paint_df[ 'real_consumption']
real_yield_difference =  paint_df['real_yield'] - calculated_real_yield

#Checa si hay valores mayor a 0
ry_differences = real_yield_difference[real_yield_difference > 0]
if not ry_differences.empty:
    print(differences)
else:
    print("Real Yield : No differences found.")

Real Yield : No differences found.


Aquí, se generó una nueva variable en donde almacena la diferencia entre rendimiento real (proporcionado en la bs) y el rendimiento std (utilizando el promedio). Igual que el código anterior si encontró dato mayor a 0 los imprime, de no ser así imprime un mensaje comentando que no hay diferencias.

En este resultado podemos ver como si encontró diferencias entre real y el nuuevo rendimiento std, donde varían de decimales hasta enteros.

In [11]:
#Rendimiento std
##rendimiento real menos rendimineto estandar actualizado
average_yield_difference = paint_df['real_yield'] - paint_df['new_average_yield'] 

#Checa si hay valores mayor a 0
ay_differences = average_yield_difference[average_yield_difference > 0]
if not ay_differences.empty:
    print(ay_differences)
else:
    print("Average Yield : No differences found.")

1       5.592390
4       0.447896
5       1.824107
6       0.890923
7       0.007035
         ...    
559    13.452139
560     6.048982
561    10.661915
563     8.277306
564     6.003808
Length: 245, dtype: float64


### 3.2 Resumen de producción
Primero se convirtieron los datos de tipo object del resumen de producción. Se encontrarón 10 columnas:
- input_material_code: codigo del material de entrada, **string**
- output_material_code: material de salida, **string**
- client_name: nombre de cliente, **string**
- inferior_clear_code: nombre de _clear_ inferior, **string**
- superior_clear_code:nombre de _clear_ superior, **string**
- inferior_color_code: nombre de color inferior, **string**
- superior_color_code: nombre de color superior, **string**
- superior_primer_code: nombre de _primer_ superior, **string**
- inferior_primer_code: nombre de _primer_ inferior, **string**
- route: ruta a tomar, **string**

In [12]:
production_df['input_material_code'] = production_df['input_material_code'].astype('string')
production_df['output_material_code'] = production_df['output_material_code'].astype('string')
production_df['client_name'] = production_df['client_name'].astype('string')
production_df['inferior_clear_code'] = production_df['inferior_clear_code'].astype('string')
production_df['superior_clear_code'] = production_df['superior_clear_code'].astype('string')
production_df['inferior_color_code'] = production_df['inferior_color_code'].astype('string')
production_df['superior_color_code'] = production_df['superior_color_code'].astype('string')
production_df['superior_primer_code'] = production_df['superior_primer_code'].astype('string')
production_df['inferior_primer_code'] = production_df['inferior_primer_code'].astype('string')
production_df['route'] = production_df['route'].astype('string')
production_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16026 entries, 0 to 17648
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   input_material_code   16026 non-null  string        
 1   output_material_code  16026 non-null  string        
 2   start_date            16026 non-null  datetime64[ns]
 3   end_date              16026 non-null  datetime64[ns]
 4   client_name           16026 non-null  string        
 5   inferior_clear_code   5829 non-null   string        
 6   superior_clear_code   1859 non-null   string        
 7   width1_mm             16026 non-null  float64       
 8   width2_mm             16026 non-null  float64       
 9   width3_mm             16026 non-null  float64       
 10  width_mm              16026 non-null  float64       
 11  thickness1_mm         16026 non-null  float64       
 12  thickness2_mm         16026 non-null  float64       
 13  thickness3_mm        

#### 3.2.1 Promedio de anchos

En el resultado podemos observar como si llego a detectar diferencias dentro del promedio del espesor, la cantidad de diferencias salió alta debido al margen de diferencia de los decimales. Al checar con un valor mayor a 0 sale un margen alto de datos.

In [13]:
# Calculate average width
average_width_mm = production_df[['width1_mm', 'width2_mm', 'width3_mm']].mean(axis=1)
#print(average_width_mm)
# Calculate real width difference
real_width_mm = production_df['width_mm'] - average_width_mm

# Check for differences
aw_differences = real_width_mm[real_width_mm >  0 ]                                                                                              
if not aw_differences.empty:
    print(aw_differences)
else:
    print("No Differences")

52       0.003333
98       0.000333
246      0.666667
253      0.666667
632      0.003333
           ...   
16096    0.000333
16097    0.000333
16546    0.003333
16547    0.003333
16548    0.003333
Length: 166, dtype: float64


Si nos vamos a diferencia mayor a 1, disminuye el margen de diferencias. 

In [14]:
# Check for differences
aw_differences = real_width_mm[real_width_mm >  1 ]
if not aw_differences.empty:
     print('Differences found: \n' , aw_differences)
else:
    print("No Differences")

Differences found: 
 1841    3.333333
dtype: float64


#### 3.2.2 Promedio de espesores

En el resultado podemos observar como si llego a detectar diferencias dentro del promedio del espesor, la cantidad de diferencias salió alta debido al margen de diferencia de los decimales. Al checar con un valor mayor a 0 sale un margen alto de datos.

In [15]:
average_thickness_mm =production_df[['thickness1_mm', 'thickness2_mm', 'thickness3_mm']].mean(axis=1)

difference_thickness_mm =  production_df['thickness1_mm'] - average_thickness_mm 

# Check for differences
t_differences = difference_thickness_mm[difference_thickness_mm >  0]
if not t_differences.empty:
     print('Differences found: \n' , t_differences)
else:
    print("No Differences")

Differences found: 
 101      0.001333
102      0.001333
103      0.000667
104      0.001000
105      0.002667
           ...   
17638    0.000333
17639    0.000333
17640    0.000333
17641    0.000333
17642    0.000333
Length: 3131, dtype: float64


Si nos vamos a diferencia mayor a 1, disminuye el margen de diferencias. 

In [16]:
# Check for differences
t_differences = difference_thickness_mm[difference_thickness_mm >  1]
if not t_differences.empty:
    print('Differences found: \n' , t_differences)
else:
    print("No Differences")

No Differences


#### 3.2.3 Colores repetidos

Verificamos si es que tenemos renglones/registros en dónde aparezcan los mismos colores. Esto es en las columnas "Inferior Clear Code"," Superior Clear Code",  "Clear Inferior",  "Clear superior", "Color Inferior",  "Color Superior", "Primer Superior" y "Primer Inferior".

Esa verificación fue llevada a cabo al filtrar el dataframe para solo quedarnos con renglones/registros en donde existan pinturas repetidas, se ignoraron los valores None en cada renglon/Registro.

Al tener el dataframe filtrado procedemos a obtener información relevante de los procesos de pintado como sacar las tipos de pinturas más utilizadas en cada proceso del pintado; estás siendo clear, superior y color


In [17]:
#Columnas para evaluar pinturas repetidas
relevant_columns = ['inferior_clear_code', 'superior_clear_code', 'inferior_color_code', 'superior_color_code', 'superior_primer_code', 'inferior_primer_code']

#Creamos copia de producción
filtered_df = production_df.copy()

# Función para verificar si hay valores repetidos en un registro específico                                                                                                                                 
def check_duplicate_colors(row):
    values = [] # se crea una lista de valores en las columnas
    for col_name in relevant_columns: #revisamos las columnas relevantes                                                                                                                                                                                                                           
        if(row[col_name] != None): # si la columna tiene un identificador de pintura
            values.append(row[col_name]) #lo agregamos a la lista

    # si todos los valores son unicos y la lista no esta vacía, significa que se llevaron a cabo procesos de pintado y cada uno utilizo pinturas distintas
    return len(values) != len(set(values)) and len(values) > 0

# Utilizamos la función con el dataframe filtrado
duplicate_indices = filtered_df[filtered_df.apply(check_duplicate_colors, axis=1)].index.tolist()

# Imprimir indices de renglones con valroes repetidos
#print("Indices of rows with repeated values in 'inferior_color_code', 'superior_color_code', 'superior_primer_code', and 'inferior_primer_code':", duplicate_indices)

# Renglones con valores repetidos en las columnas de interes
rows_with_repeated_values = filtered_df.loc[duplicate_indices, ['inferior_clear_code', 'superior_clear_code', 'inferior_color_code', 'superior_color_code', 'superior_primer_code', 'inferior_primer_code']]

rows_with_repeated_values

,inferior_clear_code,superior_clear_code,inferior_color_code,superior_color_code,superior_primer_code,inferior_primer_code
0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,<NA>,<NA>,0121-BLANCO STD KYNAR,0226-BREATHTAKING BLUE FC,0038-PRIMER 917,0038-PRIMER 917
2,<NA>,<NA>,0121-BLANCO STD KYNAR,0226-BREATHTAKING BLUE FC,0038-PRIMER 917,0038-PRIMER 917
3,<NA>,<NA>,0121-BLANCO STD KYNAR,0226-BREATHTAKING BLUE FC,0038-PRIMER 917,0038-PRIMER 917
4,<NA>,<NA>,0121-BLANCO STD KYNAR,0226-BREATHTAKING BLUE FC,<NA>,<NA>
...,...,...,...,...,...,...
17644,0028-CLEAR P/ESPUMA,<NA>,<NA>,0104-BLANCO SUPERMATIC,0021-UNIVERSAL PRIMER,<NA>
17645,0028-CLEAR P/ESPUMA,<NA>,<NA>,0104-BLANCO SUPERMATIC,0021-UNIVERSAL PRIMER,<NA>
17646,0028-CLEAR P/ESPUMA,<NA>,<NA>,0104-BLANCO SUPERMATIC,0021-UNIVERSAL PRIMER,<NA>
17647,0028-CLEAR P/ESPUMA,<NA>,<NA>,0104-BLANCO SUPERMATIC,0021-UNIVERSAL PRIMER,<NA>


Información de pinturas existentes en los procesos

In [18]:
# Inicializamos diccionarios para almacenar las pinturas únicas por proceso
unique_paints_by_process = {
    'clear': set(),
    'color': set(),
    'primer': set()
}

# Iteramos sobre cada columna para identificar las pinturas únicas y su proceso correspondiente
for col in rows_with_repeated_values.columns:
    if 'clear' in col:
        unique_paints_by_process['clear'].update(rows_with_repeated_values[col].dropna().unique())
    elif 'color' in col:
        unique_paints_by_process['color'].update(rows_with_repeated_values[col].dropna().unique())
    elif 'primer' in col:
        unique_paints_by_process['primer'].update(rows_with_repeated_values[col].dropna().unique())

# Convertir sets a listas para una manipulación más fácil si es necesario
for process in unique_paints_by_process:
    unique_paints_by_process[process] = list(unique_paints_by_process[process])

# Mostrar las pinturas únicas por proceso
for process, paints in unique_paints_by_process.items():
    print(f"Proceso de {process}:")
    for paint in paints:
        print(f" - {paint}")

# Para obtener una lista general de todas las pinturas únicas, sin importar el proceso
all_unique_paints = set().union(*unique_paints_by_process.values())
print("\nLista general de todas las pinturas únicas:")
for paint in all_unique_paints:
    print(f" - {paint}")

# Determinar en qué proceso(s) se utiliza cada pintura única
paints_process_usage = {paint: [] for paint in all_unique_paints}
for paint in all_unique_paints:
    for process, paints in unique_paints_by_process.items():
        if paint in paints:
            paints_process_usage[paint].append(process)

# Mostrar en qué proceso(s) se utiliza cada pintura
print("\nUso de pinturas en procesos:")
for paint, processes in paints_process_usage.items():
    print(f" - Pintura {paint} se utiliza en el(los) proceso(s) de {' y '.join(processes)}.")


Proceso de clear:
 - 1055-CLEAR TOP LAVADORAS
 - 0068-APPLIANCE 10 GLOSS CLEAR
 - 0027-80 GLOSS CLEAR BACKER II
 - 0093-50 GLOSS CLEAR ONE PASS
 - 0051-80 GLOSS CLEAR BACKER.
 - 0036-BACK PRIMER
 - 1078-CLEAR TOP 20 GLOSS
 - 1022-EPOXY CHROMATE BACKER
 - 0099-CLEAR P/ESPUMA ROHS
 - 0097-CLEAR APPLIANCE BACKER MC
 - 0025-CLEAR EPOXY (WASH COAT)
 - 1026-CLEAR PRINT INOX
 - 0744-AMARILLO TERNIUM CR FC
 - 0066-SUPER HIGH GLOSS CLEAR II
 - 0056-50 GLOSS APPLIANCE CLEAR
 - 0034-CLEAR AZUL P/ESPUMA
 - 0028-CLEAR P/ESPUMA
 - 1028-CLEAR BLUE EPOXY
 - 0054-CLEAR KRYSTAL KOTE
 - 1009-APPLIANCE 30 GLOSS CLR OP
 - 0023-CLEAR EPOXICO P/ESPUMA
 - 0024-CLEAR LAUNDRY BACKER
 - 0058-SUPER HIGH GLOSS CLEAR
 - 1077-CLEAR PRINT INOX MATE
 - 0029-CLEAR INOXIDABLE
 - 0075-CLEAR LAUNDRY BACKER ROHS
 - 0063-CLEAR KYNAR UV
 - 1070-CLEAR P/ESPUMA GRIS
 - 0092-80 GLOSS CLEAR LAV
 - 1152-BLANCO TERNIUM CR TOP
Proceso de color:
 - 0966-ROJO RAL 8004 BC
 - 1492-GRIS KB LAV
 - 0933-ROJO RAL-8004 POLY
 - 0524-BLACK KR

Información sobre colores repetidos en cada tipo de proceso

In [19]:
# Definimos diccionarios para agrupar los identificadores de pinturas por tipo de proceso
clear_codes = {'inferior_clear_code', 'superior_clear_code'}
primer_codes = {'inferior_primer_code', 'superior_primer_code'}
color_codes = {'inferior_color_code', 'superior_color_code'}

# Creamos DataFrames vacíos para cada tipo de proceso
clear_paints = pd.DataFrame(columns=['Paint Code', 'Process Type', 'Count'])
primer_paints = pd.DataFrame(columns=['Paint Code', 'Process Type', 'Count'])
color_paints = pd.DataFrame(columns=['Paint Code', 'Process Type', 'Count'])

# Función para contar y agregar pinturas por tipo de proceso
def add_paint_count(row, process_dict, process_name, df):
    for col_name in process_dict:
        paint_code = row[col_name]
        if pd.notna(paint_code):  # Verificamos que el código de pintura no sea NaN
            if not df[df['Paint Code'] == paint_code].empty:
                df.loc[df['Paint Code'] == paint_code, 'Count'] += 1
            else:
                df = pd.concat([df, pd.DataFrame([[paint_code, process_name, 1]], columns=df.columns)], ignore_index=True)
    return df

# Recorremos el DataFrame para contar y agrupar pinturas por proceso
for index, row in rows_with_repeated_values.iterrows():
    clear_paints = add_paint_count(row, clear_codes, 'Clear', clear_paints)
    primer_paints = add_paint_count(row, primer_codes, 'Primer', primer_paints)
    color_paints = add_paint_count(row, color_codes, 'Color', color_paints)

# Unimos los resultados para tener un resumen completo
all_paints = pd.concat([clear_paints, primer_paints, color_paints]).reset_index(drop=True)

# Mostramos las pinturas existentes y su uso por proceso
print("Summary of Paint Use by Process:")
#print(all_paints.sort_values(by=['Process Type', 'Count'], ascending=[True, False]))

# Para ver cuáles pinturas son las más usadas en cada proceso
print("\nMost Used Paints by Process:")
for process_type, group_df in all_paints.groupby('Process Type'):
    most_used = group_df.loc[group_df['Count'].idxmax()]
    print(f"{process_type}: {most_used['Paint Code']} used {most_used['Count']} times")

all_paints.sort_values(by=['Process Type', 'Count'], ascending=[True, False])

Summary of Paint Use by Process:

Most Used Paints by Process:
Clear: 0028-CLEAR P/ESPUMA used 2365 times
Color: 0601-VERDE PRIMSA used 3764 times
Primer: 0001-PRIMER 4457 used 3665 times


,Paint Code,Process Type,Count
3,0028-CLEAR P/ESPUMA,Clear,2365
4,0054-CLEAR KRYSTAL KOTE,Clear,1069
5,0023-CLEAR EPOXICO P/ESPUMA,Clear,978
11,0024-CLEAR LAUNDRY BACKER,Clear,816
17,0097-CLEAR APPLIANCE BACKER MC,Clear,577
...,...,...,...
47,1030-PRIMER 4494,Primer,8
42,0018-HIGH BUILT PRIMER...,Primer,5
46,1043-WHITE PRIMER LAUNDRY ROHS,Primer,4
49,0064-PO PRIMER 6334A,Primer,2


## 4 .Valores faltantes

### 4.1 Suma de nulos por variable 

#### 4.1.1 Análisis de consumo de pintura

En el resultado podemos observar cómo no existe algún valor nulo dentro de las variables que seleccionamos para esta base de datos, estos son datos que son necesarios dentro del proceso de pintado.

In [20]:
# Calcular el total de valores nulos para cada columna
paint_null_values= paint_df.isnull().sum()

print("paint_df , Null values per column:")                             
paint_null_values

paint_df , Null values per column:


production_line                0
month                          0
month_number                   0
paint                          0
real_consumption               0
theoretical_consumption        0
consumption_difference         0
average_yield                  0
real_yield                     0
yield_difference               0
real_produced_square_meters    0
new_average_yield              0
dtype: int64

#### 4.1.2 Pinturas y revestidos

En el resultado podemos observar cómo no existe algún valor nulo dentro de las variables que seleccionamos para esta base de datos, estos son datos que son necesarios en la evaluación del proceso de las pinturas. En donde se identifica datos importantes de la pintura utilizada como proveedor y precio.

In [21]:
# Calcular el total de valores nulos para cada columna
coating_null_values= coating_df.isnull().sum()

print("coating_df, Null values per column:")
coating_null_values                                                            

coating_df, Null values per column:


production_line       0
paint_id              0
paint_name            0
monetary_value_usd    0
total_liters_used     0
production_plant      0
supplier              0
date                  0
hour                  0
price_per_liter       0
dtype: int64

#### 4.1.3 Resumen de producción

En el resultado podemos observar cómo aquí si existen valores nulos en algunas variables de la base de datos. Esto puede llegar a pasar porque no se realizó el proceso necesario o hasta no se guardó el dato usado.

In [22]:
# Calcular el total de valores nulos para cada columna
production_null_values= production_df.isnull().sum()

print("production_df, Null values per column:")
production_null_values

production_df, Null values per column:


input_material_code         0
output_material_code        0
start_date                  0
end_date                    0
client_name                 0
inferior_clear_code     10197
superior_clear_code     14167
width1_mm                   0
width2_mm                   0
width3_mm                   0
width_mm                    0
thickness1_mm               0
thickness2_mm               0
thickness3_mm               0
thickness_mm                0
input_weight_kg             0
weight_kg                   0
length_m                    0
inferior_color_code     10999
superior_color_code      5338
superior_primer_code    10297
inferior_primer_code    13059
route                       0
dtype: int64

#### 4.1.4 Defectos de producción

En este resultado podemos ver como la mayoría de las columnas tienen nulos, esto se debe a que las pinturas no fueron detectadas con algún defecto dentro del proceso de la revisión.

In [23]:
# Calcular el total de valores nulos para cada columna
defects_null_values= defects_df.isnull().sum()

print("defects_df, null values per column:")
defects_null_values

defects_df, null values per column:


defect_code          0
defect_name          0
location           313
is_containment      78
is_preventive       78
intensity         1639
face              1769
side              1575
frequency         1847
register_date        0
dtype: int64

### 4.2 Indentificacion de columnas con mayor porcentaje de 15% datos nulos

El código calcula primero las filas del dataframe, después calcula el porcentaje de los valores nulos por cada columna y finalmente filtra los datos con el porcentaje seleccionado (en este caso 15%).  Si identifica valores mayores a 15% siendo nulos, te imprime las columnas, de no ser así imprime un mensaje comentando que no hay nulos arriba del %15.

#### 4.2.1 Análisis de consumo de pintura

In [24]:
total = paint_df.shape[0]

null_values_percentage= (paint_df.isnull().sum() / total) * 100

total_nulls= null_values_percentage[null_values_percentage > 15]
if not total_nulls.empty:
    print("Columns with more than 15% null data:" ,total_nulls)
else:
    print("No columns above 15 percent null data")

No columns above 15 percent null data


#### 4.2.2 Pinturas y revestidos

In [25]:
total = coating_df.shape[0]

null_values_percentage= (coating_df.isnull().sum() / total) * 100

total_nulls= null_values_percentage[null_values_percentage > 15]
if not total_nulls.empty:
    print("Columns with more than 15% null data:" ,total_nulls)
else:
    print("No columns above 15 percent null data")

No columns above 15 percent null data


#### 4.2.3 Resumen de producción

In [26]:
total = production_df.shape[0]

null_values_percentage= (production_df.isnull().sum() / total) * 100

total_nulls= null_values_percentage[null_values_percentage > 15]
if not total_nulls.empty:
    print("Columns with more than 15% null data: \n" ,total_nulls)
else:
    print("No columns above 15 percent null data ")

Columns with more than 15% null data: 
 inferior_clear_code     63.627855
superior_clear_code     88.400100
inferior_color_code     68.632223
superior_color_code     33.308374
superior_primer_code    64.251841
inferior_primer_code    81.486335
dtype: float64


#### 4.2.4 Defectos de producción

In [27]:
total = defects_df.shape[0]

null_values_percentage= (defects_df.isnull().sum() / total) * 100

total_nulls= null_values_percentage[null_values_percentage > 15]
if not total_nulls.empty:
    print("Columns with more than 15% null data: \n" ,total_nulls)
else:
    print("No columns above 15 percent null data ")

Columns with more than 15% null data: 
 intensity    65.324831
face         70.506178
side         62.774014
frequency    73.614986
dtype: float64


## 5 Registros duplicados

Para cada una de los dataframes se utilizó la función de duplicated para identificar las filas duplicadas en cada uno de ellos.

#### 5.1 Resumen de producción

In [28]:
production_df[production_df.duplicated()]

,input_material_code,output_material_code,start_date,end_date,client_name,inferior_clear_code,superior_clear_code,width1_mm,width2_mm,width3_mm,...,thickness3_mm,thickness_mm,input_weight_kg,weight_kg,length_m,inferior_color_code,superior_color_code,superior_primer_code,inferior_primer_code,route


#### 5.2 Análisis de consumo de pintura

In [29]:
paint_df[paint_df.duplicated()]

,production_line,month,month_number,paint,real_consumption,theoretical_consumption,consumption_difference,average_yield,real_yield,yield_difference,real_produced_square_meters,new_average_yield


#### 5.3 Pinturas y revestidos

In [30]:
defects_df[defects_df.duplicated(keep=False)]                       

,defect_code,defect_name,location,is_containment,is_preventive,intensity,face,side,frequency,register_date
830,40.0,PESO (-),TODO,1.0,1.0,NaN,None,None,None,2023-05-30 20:12:00
831,40.0,PESO (-),TODO,1.0,1.0,NaN,None,None,None,2023-05-30 20:12:00
864,40.0,PESO (-),TODO,1.0,1.0,NaN,None,None,None,2023-06-06 16:08:00
865,40.0,PESO (-),TODO,1.0,1.0,NaN,None,None,None,2023-06-06 16:08:00
918,40.0,PESO (-),TODO,0.0,0.0,NaN,None,None,None,2023-06-16 01:56:00
919,40.0,PESO (-),TODO,0.0,0.0,NaN,None,None,None,2023-06-16 01:56:00
1350,647.0,CINTA DE NOCHE,None,NaN,NaN,NaN,None,None,None,2023-07-14 05:39:00
1351,647.0,CINTA DE NOCHE,None,NaN,NaN,NaN,None,None,None,2023-07-14 05:39:00
1699,674.0,MANCHAS SIN GALVANIZAR,TODO,0.0,0.0,3.0,AMBAS,AMBOS LADOS,CONTINUA,2023-08-02 19:25:00
1700,674.0,MANCHAS SIN GALVANIZAR,TODO,0.0,0.0,3.0,AMBAS,AMBOS LADOS,CONTINUA,2023-08-02 19:25:00


In [31]:
coating_df[coating_df.duplicated()]       

,production_line,paint_id,paint_name,monetary_value_usd,total_liters_used,production_plant,supplier,date,hour,price_per_liter
508,Pintado 1 Prod. Univ,I1248_VALS,0054-CLEAR KRYSTAL KOTE,1882.00,200.00,Uni,VALS,24.07.2020,23:16:33,9.410000
509,Pintado 1 Prod. Univ,I1248_VALS,0054-CLEAR KRYSTAL KOTE,1882.00,200.00,Uni,VALS,24.07.2020,23:16:33,9.410000
510,Pintado 1 Prod. Univ,I1248_VALS,0054-CLEAR KRYSTAL KOTE,1882.00,200.00,Uni,VALS,24.07.2020,23:16:33,9.410000
511,Pintado 1 Prod. Univ,I1248_VALS,0054-CLEAR KRYSTAL KOTE,1882.00,200.00,Uni,VALS,24.07.2020,23:16:33,9.410000
531,Pintado 1 Prod. Univ,I4461_VALS,0435-APOLLO GRAY KRYSTAL KOTE,1856.00,200.00,Uni,VALS,28.07.2020,03:41:16,9.280000
...,...,...,...,...,...,...,...,...,...,...
211698,Pintado 1 UNI,I0402_VALS,0400-GRIS FONDO,-423.00,-75.00,Uni,VALS,18.08.2023,00:00:00,5.640000
211699,Pintado 1 UNI,I4078_VALS,WHITE KRYSTAL KOTE,-896.00,-70.00,Uni,VALS,18.08.2023,00:00:00,12.800000
211701,Pintado 1 UNI,I0100_VALS,0102-BLANCO STD,-1170.00,-200.00,Uni,VALS,18.08.2023,00:00:00,5.850000
211704,Pintado 1 UNI,I1270_BECK,0079-BECKRYPRIM 246,-1470.00,-200.00,Uni,BECK,18.08.2023,00:00:00,7.350000


## Contribución individual

#### Hiram Muñoz
Construcción del _script_ de procesamiento de datos, verificación de tipos de datos, selección y eliminación de columnas innecesarias.
#### Raúl Murillo 
Verificación de la columna de diferencia de las columnas real y teórico y de la verificación de los metros cuadrados reales.

#### Andrea Garza 
Verificación del rendimiento real y diferencia con el rendimiento estándar, Calculo e identificación de diferencias en el promedio de Ancho y Espesor e Identificación de valores faltantes.

#### Erick Hernández 
Calculo de nuevo rendimiento std, identificación de renglones duplicados y verificación de metros cuadrados reales 

#### David Martínez
Cambio de nombres de variables y analizar información sobre pinturas repetidas en los renglones de producción (encontrados duplicados y segmentadas pinturas que aparecen por cada proceso)

#### Distribucion de Tareas: 
https://docs.google.com/spreadsheets/d/1Nx3N0PPtPqMVHnxO9pkpvJUUQX7RMOTvP8N3Cbafq70/edit?usp=sharing 

## Referencias
Apache Software Foundation. (s. f.). Feather File Format — Apache Arrow v15.0.1. Recuperado 12 de marzo de 2024, de https://arrow.apache.org/docs/python/feather.html